---

_You are currently looking at **version 1.2** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-social-network-analysis/resources/yPcBs) course resource._

---

# Assignment 2 - Network Connectivity

In this assignment you will go through the process of importing and analyzing an internal email communication network between employees of a mid-sized manufacturing company. 
Each node represents an employee and each directed edge between two nodes represents an individual email. The left node represents the sender and the right node represents the recipient.

In [1]:
import networkx as nx

# This line must be commented out when submitting to the autograder
#!head email_network.txt

### Question 1

Using networkx, load up the directed multigraph from `email_network.txt`. Make sure the node names are strings.

*This function should return a directed multigraph networkx graph.*

In [2]:
def answer_one():
    
    DM = nx.read_edgelist('email_network.txt', nodetype=str, data=[('time',int)], create_using=nx.MultiDiGraph())
    
    return DM

### Question 2

How many employees and emails are represented in the graph from Question 1?

*This function should return a tuple (#employees, #emails).*

In [3]:
def answer_two():
        
    DM = answer_one()
    # each edge is an email
    emails = len(DM.edges())
    # each node is an employee
    employees = len(DM.nodes())
    
    return (employees, emails)

In [4]:
answer_two()

(167, 82927)

### Question 3

* Part 1. Assume that information in this company can only be exchanged through email.

    When an employee sends an email to another employee, a communication channel has been created, allowing the sender to provide information to the receiver, but not vice versa. 

    Based on the emails sent in the data, is it possible for information to go from every employee to every other employee?


* Part 2. Now assume that a communication channel established by an email allows information to be exchanged both ways. 

    Based on the emails sent in the data, is it possible for information to go from every employee to every other employee?


*This function should return a tuple of bools (part1, part2).*

In [5]:
def answer_three():
        
    # part 1 = is the directed multigraph strongly connected?
    # part 2 = is the directed multigraph weakly connected?
    DM = answer_one()
    
    return (nx.is_strongly_connected(DM), nx.is_weakly_connected(DM))

In [6]:
answer_three()

(False, True)

### Question 4

How many nodes are in the largest (in terms of nodes) weakly connected component?

*This function should return an int.*

In [7]:
def answer_four():
        
    # As the graph is weakly connected, the largest weakly connected component is the whole graph.
    # Therefore, the answer will be equal to the number of nodes in the graph (167).
    # But let's use a more robust way to find the answer:
    DM = answer_one()
    
    weakly_conn_components = nx.weakly_connected_components(DM) #this is a generator of the components
    # to get the list of the components use: sorted(weakly_conn_components)
    # in this case there is only 1 weakly connected component
    length_of_w_components = [len(c) for c in sorted(weakly_conn_components)]
    
    return max(length_of_w_components)

In [8]:
answer_four()

167

### Question 5

How many nodes are in the largest (in terms of nodes) strongly connected component?

*This function should return an int*

In [9]:
def answer_five():
        
    DM = answer_one()
    
    strongly_conn_components = nx.strongly_connected_components(DM) #this is a generator of the components
    # to get the list of the components use: sorted(strongly_conn_components)
    length_of_s_components = [len(c) for c in sorted(strongly_conn_components)]
    
    return max(length_of_s_components)

In [10]:
answer_five()

126

### Question 6

Using the NetworkX function strongly_connected_component_subgraphs, find the subgraph of nodes in a largest strongly connected component. 
Call this graph G_sc.

*This function should return a networkx MultiDiGraph named G_sc.*

In [11]:
def answer_six():
    
    DM = answer_one()
    G_sc = max(nx.strongly_connected_component_subgraphs(DM), key=len)
    
    return G_sc

In [12]:
answer_six()

### Question 7

What is the average distance between nodes in G_sc?

*This function should return a float.*

In [13]:
def answer_seven():
        
    G_sc = answer_six()
    
    return nx.average_shortest_path_length(G_sc)

In [14]:
answer_seven()

1.6461587301587302

### Question 8

What is the largest possible distance between two employees in G_sc?

*This function should return an int.*

In [15]:
def answer_eight():
        
    G_sc = answer_six()
    
    return nx.diameter(G_sc)

In [16]:
answer_eight()

3

### Question 9

What is the set of nodes in G_sc with eccentricity equal to the diameter?

*This function should return a set of the node(s).*

In [17]:
def answer_nine():
    
    # DISTANCE between node N and M: shortest path between them
    # ECCENTRICITY of node N is the maximum of the distances from node N to each of the other nodes
    # DIAMETER of the graph is the maximum of all the eccentricities
    # RADIUS of the graph is the minimum of all the eccentricities
    # PERIPHERY of the graph: set of nodes with eccentricity=diameter
    # CENTER of the graph: set of nodes with eccentricity=radius
    
    G_sc = answer_six()

    return set(nx.periphery(G_sc))   #nx.periphery(G_sc) returns a list

In [18]:
answer_nine()

{'129', '134', '97'}

### Question 10

What is the set of node(s) in G_sc with eccentricity equal to the radius?

*This function should return a set of the node(s).*

In [19]:
def answer_ten():
        
    G_sc = answer_six()

    return set(nx.center(G_sc))

In [20]:
answer_ten()

{'38'}

### Question 11

Which node in G_sc is connected to the most other nodes by a shortest path of length equal to the diameter of G_sc?

How many nodes are connected to this node?


*This function should return a tuple (name of node, number of satisfied connected nodes).*

In [21]:
def answer_eleven():
        
    G_sc = answer_six()
    diameter = nx.diameter(G_sc)   #integer 3
    peripheries = nx.periphery(G_sc)   #list ['134', '129', '97']
    max_count = 0
    result_node = None
    for node in peripheries:
        count = 0
        # dict with keys all the nodes but 'node' and value equal to the distance between 'node' and the key node
        sp = nx.shortest_path_length(G_sc, source=node)
        for key, value in sp.items():
            if value == diameter:
                count += 1
            if count >= max_count:
                result_node = node
                max_count = count
    
    return (result_node, max_count)

In [22]:
answer_eleven()

('97', 63)

### Question 12

Suppose you want to prevent communication from flowing to the node that you found in the previous question from any node in the center of G_sc, what is the smallest number of nodes you would need to remove from the graph (you're not allowed to remove the node from the previous question or the center nodes)? 

*This function should return an integer.*

In [23]:
def answer_twelve():
    
    # There's only 1 node in the center of the graph (node '38') 
    # But let's use a more robust way to find the answer:
    G_sc = answer_six()
    
    target_node = answer_eleven()[0]
    center_nodes_list = nx.center(G_sc)
    
    # list of nodes to cut
    nodes_to_cut = []
    
    for c in center_nodes_list:
        c_nodes_to_cut = list(nx.minimum_node_cut(G_sc, s=center_nodes_list[0], t=target_node)) #convert the set to a list
        #extend() is like append() but intead of appending list2 to list1, it appends THE ELEMENTS of list2 to list1
        nodes_to_cut.extend(c_nodes_to_cut) 
        
    # remove duplicate nodes from nodes_to_cut list
    nodes_to_cut = list(set(nodes_to_cut))
    
    return len(nodes_to_cut)

In [24]:
answer_twelve()

5

### Question 13

Construct an undirected graph G_un using G_sc (you can ignore the attributes).

*This function should return a networkx Graph.*

In [25]:
def answer_thirteen():
        
    G_sc = answer_six()
    
    # convert directed graph to undirected
    # reciprocal=False: not necessary for edges to exist in both directions to be kept in G_un
    G_un = G_sc.to_undirected(reciprocal=False)
    
    return nx.Graph(G_un) #otherwise, the function returns a MultiGraph, not a Graph

In [26]:
answer_thirteen()

### Question 14

What is the transitivity and average clustering coefficient of graph G_un?

*This function should return a tuple (transitivity, avg clustering).*

In [27]:
def answer_fourteen():
        
    G_un = answer_thirteen()
    
    transitivity = nx.transitivity(G_un)
    avg_clustering_coef = nx.average_clustering(G_un)
    
    return (transitivity, avg_clustering_coef)

In [28]:
answer_fourteen()

(0.570111160700385, 0.6975272437231421)